In [ ]:
#Getting started with Pandas: dataframe and stuff
import csv as csv
import numpy as np

csv_file_object = csv.reader(open('C:/Users/uf425c/Documents/Python Projects/Titanic Survivor/Data/train.csv', 'rb')) 
header = csv_file_object.next() 
data=[] 

for row in csv_file_object:
    data.append(row)
data = np.array(data) 

data.head(5)

In [9]:
import pandas as pd
import numpy as np
df = pd.read_csv('C:/Users/uf425c/Documents/Python Projects/Titanic Survivor/Data/train.csv', header=0)

type(df)
df.dtypes

df.head(3)
df[ df['Name'].str.contains('Mr.') | df['Name'].str.contains('Miss') ].head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Prefix
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,1
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,1
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S,1
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,NaN,S,2
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,NaN,C,2
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,PP 9549,16.7000,G6,S,3


In [4]:
#df[df['Age'].isnull()].head(10)
#df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
df['AgeFill'] = df['Age']
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Prefix,Gender,AgeFill
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,1,1,22
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,2,0,38


In [5]:
median_ages = np.zeros((3,3))

for i in range(0, 3):
    for j in range(0, 3):
        median_ages[i,j] = df[(df['Prefix'] == i+1) & \
                              (df['Pclass'] == j+1)]['Age'].dropna().median()
        
median_ages

array([[ 40. ,  31. ,  26. ],
       [ 41.5,  31.5,  31. ],
       [ 30. ,  24. ,  18. ]])

In [6]:
for i in range(0, 3):
    for j in range(0, 3):
        df.loc[ (df.Age.isnull()) & (df.Prefix == i+1) & (df.Pclass == j+1),\
                'AgeFill'] = median_ages[i,j]

In [7]:
df[ (df['Name'].str.contains('Mr.') | df['Name'].str.contains('Miss') | df['Name'].str.contains('Ms')) 
   & df['Age'].isnull()].head()

#df[ (df['Name'].str.contains('Mr.') | df['Name'].str.contains('Miss') | df['Name'].str.contains('Ms')) 
#   & df['Age'].notnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Prefix,Gender,AgeFill
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,1,1,26
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S,1,1,31
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C,2,0,31
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C,1,1,26
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q,3,0,18


In [ ]:
#Embarked
pd.get_dummies(df['Embarked'], prefix='Embarked').head(10)
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')], axis=1)

In [ ]:
#Age
df[ df['AgeFill'].isnull() ][['Gender','Pclass','Age','AgeFill']].head(10)

for i in range(0, 2):
    for j in range(0, 3):
        df.loc[ (df.AgeFill.isnull()) & (df.Gender == i) & (df.Pclass == j+1),\
                'AgeFill'] = median_ages[i,j]

In [ ]:
#Fare
fare_means = df.pivot_table('Fare', index='Pclass', aggfunc='mean')
df['Fare'] = df[['Fare', 'Pclass']].apply(lambda x:
                            fare_means[x['Pclass']] if pd.isnull(x['Fare'])
                            else x['Fare'], axis=1)

In [ ]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df = df.drop(['Sex', 'Embarked', 'Age', 'Prefix'], axis=1)
df.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]

df = df[cols]

train_data = df.values

model = RandomForestClassifier(n_estimators = 100, max_features=0.5, max_depth=5.0)
#train data
model = model.fit(train_data[0:,2:], train_data[0:,0])

In [ ]:
df['Age'].mean()

In [ ]:
df['Age'][0:10] 

In [ ]:
df[df['Age'] > 65][['Sex', 'Pclass', 'Age', 'Survived']]


In [ ]:
for i in range(1,4):
    print i, len(df[ (df['Sex'] == 'male') & (df['Pclass'] == i) ])

In [ ]:
import pylab as P
df['Age'].hist()

df['Age'].dropna().hist(bins=16, range=(0,80), alpha = .5, color = 'red')
P.show()

In [ ]:
df['Gender'] = 4
df.head()

In [ ]:
df['Gender'] = df['Sex'].map( lambda x: x[0].upper() )
df.head()

In [ ]:
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
df.head()

In [ ]:
median_ages = np.zeros((2,3))

for i in range(0, 2):
    for j in range(0, 3):
        median_ages[i,j] = df[(df['Gender'] == i) & \
                              (df['Pclass'] == j+1)]['Age'].dropna().median()
 
median_ages

In [ ]:
df['AgeFill'] = df['Age']

df.head()

In [ ]:
for i in range(0, 2):
    for j in range(0, 3):
        df.loc[ (df.Age.isnull()) & (df.Gender == i) & (df.Pclass == j+1),\
                'AgeFill'] = median_ages[i,j]
        
df[ df['Age'].isnull() ][['Gender','Pclass','Age','AgeFill']].head(10)

In [ ]:
df[ df['Age'].isnull() ].head(10)

In [ ]:
df['FamilySize'] = df['SibSp'] + df['Parch']

df.head(5)

In [ ]:
from scipy.stats import mode

mode_embarked = mode(df['Embarked'])[0][0]
df['Embarked'] = df['Embarked'].fillna(mode_embarked)

df['Gender'] = df['Sex'].map({'female': 0, 'male': 1}).astype(int)
df['Port'] = df['Embarked'].map({'C':1, 'S':2, 'Q':3}).astype(int)

In [ ]:
df.head(5)

In [ ]:
df = df.drop(['Sex', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked'], axis=1)

In [ ]:
df.dtypes[df.dtypes.map(lambda x: x=='object')]

In [ ]:
#df = df.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1) 
#df = df.drop(['Age'], axis=1)
#df = df.dropna()
cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
df = df[cols]

In [ ]:
train_data = df.values
train_data

In [ ]:
#df = df.dropna(subset=['Embarked']) 
df = df.dropna()

In [ ]:
df[ df['Sex'].isnull() ].head(10)
df[ df['Embarked'].isnull() ].head(10)


In [ ]:
df['Gender'] = df['Sex'].map({'female': 0, 'male':1}).astype(int)
df['Port'] = df['Embarked'].map({'C':1, 'S':2, 'Q':3}).astype(int)


In [ ]:
df = df.drop(['Age'], axis=1)

In [ ]:
df.head(5)
cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
df = df[cols]
#df[ df['Name'].str.contains('Dr.')].head(10)

In [ ]:
df.info()
train_data = df.values
train_data


In [14]:
#Get training
df = pd.read_csv('C:/Users/uf425c/Documents/Python Projects/Titanic Survivor/Data/train.csv', header=0)

df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

age_mean = df['Age'].mean()
df['Age'] = df['Age'].fillna(age_mean)

from scipy.stats import mode
#mode_embarked = mode(df['Embarked'])[0][0]
#df['Embarked'] = df['Embarked'].fillna(mode_embarked)

df['Embarked'].isnull= df['Embarked'].dropna().mode().values

df['Gender'] = df['Sex'].map({'female': 0, 'male': 1}).astype(int)

pd.get_dummies(df['Embarked'], prefix='Embarked').head(10)
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')], axis=1)

df = df.drop(['Sex', 'Embarked'], axis=1)

cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]

df = df[cols]

train_data = df.values

df.head()

,Survived,PassengerId,Pclass,Age,SibSp,Parch,Fare,Prefix,Gender,Embarked_C,Embarked_Q,Embarked_S
0,0,1,3,22,1,0,7.2500,1,1,0,0,1
1,1,2,1,38,1,0,71.2833,2,0,1,0,0
2,1,3,3,26,0,0,7.9250,3,0,0,0,1
3,1,4,1,35,1,0,53.1000,2,0,0,0,1
4,0,5,3,35,0,0,8.0500,1,1,0,0,1


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

parameter_grid = {
    'max_features': [0.5, 1.],
    'max_depth': [5., None]
}

grid_search = GridSearchCV(RandomForestClassifier(n_estimators = 100), parameter_grid,
                            cv=5, verbose=2)

grid_search.fit(train_data[0:,2:], train_data[0:,0])

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] max_features=0.5, max_depth=5.0 .................................


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [16]:
grid_search.grid_scores_

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [ ]:
sorted(grid_search.grid_scores_, key=lambda x: x.mean_validation_score)
grid_search.best_score_
grid_search.best_params_

In [17]:
model = RandomForestClassifier(n_estimators = 100, max_features=.5, max_depth=12.0)

#train data
model = model.fit(train_data[0:,2:], train_data[0:,0])

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
#import test data
df_test = pd.read_csv('C:/Users/uf425c/Documents/Python Projects/Titanic Survivor/Data/test.csv', header=0)

df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

#df_test['Age'] = df_test['Age'].fillna(age_mean)

fare_means = df.pivot_table('Fare', index='Pclass', aggfunc='mean')
df_test['Fare'] = df_test[['Fare', 'Pclass']].apply(lambda x:
                            fare_means[x['Pclass']] if pd.isnull(x['Fare'])
                            else x['Fare'], axis=1)

df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)

df_test['AgeFill'] = df_test['Age']

median_ages = np.zeros((3,3))

for i in range(0, 3):
    for j in range(0, 3):
        median_ages[i,j] = df_test[(df_test['Prefix'] == i+1) & \
                              (df_test['Pclass'] == j+1)]['Age'].dropna().median()
        
for i in range(0, 3):
    for j in range(0, 3):
        df_test.loc[ (df_test.Age.isnull()) & (df_test.Prefix == i+1) & (df_test.Pclass == j+1),\
                'AgeFill'] = median_ages[i,j]

age_mean = df['AgeFill'].mean()
df_test['AgeFill'] = df_test['AgeFill'].fillna(age_mean)
        
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')],
                axis=1)

df_test = df_test.drop(['Sex', 'Embarked', 'Age', 'Prefix'], axis=1)

test_data = df_test.values

df_test.head()

In [ ]:
output = model.predict(test_data[:,1:])

In [ ]:
result = np.c_[test_data[:,0].astype(int), output.astype(int)]

In [ ]:
result = np.c_[test_data[:,0].astype(int), output.astype(int)]
df_result = pd.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])


In [ ]:
df_result.head(10)

In [ ]:
df_result = pd.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])
df_result.to_csv('C:/Users/uf425c/Documents/Python Projects/Titanic Survivor/titanic_1-3.csv', index=False)

In [ ]:
df.head()

In [ ]:
df_test.head()